# Group 2: Dekker, Hauser, Tassone, Vogel
## buy/hodl/sell

In [112]:
##### packages
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import math
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer


In [9]:
##### reading data
#AH
df14 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2014_Financial_Data.csv')
df15 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2015_Financial_Data.csv')
df16 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2016_Financial_Data.csv')
df17 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2017_Financial_Data.csv')
df18 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2018_Financial_Data.csv')


df_sp500 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\sp-500-historical-annual-returns.csv')

In [10]:
##### df manipulation
df_sp500 = df_sp500[86:91]

#add column with year
df14['year'] = 2014
df15['year'] = 2015
df16['year'] = 2016
df17['year'] = 2017
df18['year'] = 2018

#add column for recom
df14['recommendation'] = 0
df15['recommendation'] = 0
df16['recommendation'] = 0
df17['recommendation'] = 0
df18['recommendation'] = 0

#rename 'Price Var' col
df14.rename(columns={"2015 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df15.rename(columns={"2016 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df16.rename(columns={"2017 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df17.rename(columns={"2018 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df18.rename(columns={"2019 PRICE VAR [%]": "PRICE_VAR"},inplace=True)

#dropping name column
df14 = df14.drop(df14.columns[0], axis=1)
df15 = df15.drop(df15.columns[0], axis=1)
df16 = df16.drop(df16.columns[0], axis=1)
df17 = df17.drop(df17.columns[0], axis=1)
df18 = df18.drop(df18.columns[0], axis=1)

#df_all = pd.concat([df14, df15, df16, df17, df18], ignore_index=True) #for after imputation

In [54]:
##### some minor EDA
df14.columns
df14.info() #we have mostly floats, but also 3 integer variables and 1 object variable (-> see 'One-Hot-Encoding' or 'LabelEncoding')
df14['Class']
df14.describe()

#dimensions for each data set
year = 14
for el in [df14, df15, df16, df17, df18]:
    print(year, el.shape)
    year += 1

#some boxplots
#plt.boxplot(df14['PRICE_VAR'])

#plt.figure()
#df_bp_log = log(df14['Revenue'])#log scale for better visualization
#df_bp_log['Sector'] = df14['Sector']
#sns.catplot(x="Revenue", y="Sector", kind="box", data=df_bp_log)

Index(['Revenue', 'Revenue Growth', 'Cost of Revenue', 'Gross Profit',
       'R&D Expenses', 'SG&A Expense', 'Operating Expenses',
       'Operating Income', 'Interest Expense', 'Earnings before Tax',
       ...
       'Asset Growth', 'Book Value per Share Growth', 'Debt Growth',
       'R&D Expense Growth', 'SG&A Expenses Growth', 'Sector', 'PRICE_VAR',
       'Class', 'year', 'recommendation'],
      dtype='object', length=226)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808 entries, 0 to 3807
Columns: 226 entries, Revenue to recommendation
dtypes: float64(222), int64(3), object(1)
memory usage: 6.6+ MB


0       0
1       0
2       1
3       1
4       1
       ..
3803    1
3804    0
3805    0
3806    1
3807    0
Name: Class, Length: 3808, dtype: int64

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,PRICE_VAR,Class,year,recommendation
count,3.764000e+03,3572.000000,3.734000e+03,3.756000e+03,3.672000e+03,3.749000e+03,3.745000e+03,3.753000e+03,3.745000e+03,3.728000e+03,...,3518.000000,3518.000000,3439.000000,3506.000000,3561.000000,3565.000000,3.808000e+03,3808.000000,3808.0,3808.0
mean,5.879050e+09,12.954244,3.700973e+09,2.188214e+09,9.401830e+07,9.307406e+08,1.438144e+09,6.748248e+08,1.002018e+08,5.725866e+08,...,0.164692,2.279999,0.813931,1.361780,0.596529,0.382284,9.751560e+02,0.429097,2014.0,0.0
std,3.901741e+10,705.605473,3.040688e+10,1.159028e+10,6.408912e+08,4.820928e+09,7.305171e+09,3.786291e+09,6.991091e+08,2.793740e+09,...,2.195115,92.428920,25.630214,18.734301,25.894049,4.696975,4.098461e+04,0.495012,0.0,0.0
min,-6.276160e+08,-1.773200,-5.455740e+08,-1.105000e+09,-1.500000e+05,0.000000e+00,-1.088448e+09,-6.786000e+09,-2.250000e+08,-8.878000e+09,...,-1.000000,-0.979600,-230.000000,-1.051000,-1.043700,-1.000000,-1.003972e+02,0.000000,2014.0,0.0
25%,5.788880e+07,-0.002350,3.135714e+06,3.092900e+07,0.000000e+00,1.548900e+07,3.106600e+07,-1.308000e+06,0.000000e+00,-3.732902e+06,...,0.000000,-0.015400,-0.084550,-0.048200,0.000000,-0.003900,-2.857143e+01,0.000000,2014.0,0.0
50%,4.349010e+08,0.061850,1.414420e+08,1.908760e+08,0.000000e+00,7.381800e+07,1.386610e+08,4.104400e+07,2.563000e+06,2.842700e+07,...,0.000000,0.062200,0.037100,0.000000,0.000000,0.068800,-5.281842e+00,0.000000,2014.0,0.0
75%,2.393625e+09,0.188875,1.199844e+09,8.922534e+08,9.911000e+06,3.510000e+08,5.885100e+08,2.712980e+08,4.300000e+07,2.193835e+08,...,0.063850,0.217100,0.141900,0.210300,0.004700,0.220500,1.331484e+01,1.000000,2014.0,0.0
max,1.824698e+12,42138.663900,1.537249e+12,4.621600e+11,1.153700e+10,1.856830e+11,3.056050e+11,1.565540e+11,3.152300e+10,8.720500e+10,...,76.625000,5468.426400,1360.125000,729.576600,1542.611000,225.690000,2.418601e+06,1.000000,2014.0,0.0


14 (3808, 226)
15 (4120, 226)
16 (4797, 226)
17 (4960, 226)
18 (4392, 226)


101103

114348

212549

226269

97298

In [ ]:
##### check missing values
df14.isnull().sum().sum()
df15.isnull().sum().sum()
df16.isnull().sum().sum()
df17.isnull().sum().sum()
df18.isnull().sum().sum()
#df_all.isnull().sum().sum()

In [40]:
##### further exploration regarding NAs
#function to give back the colnames of cols with more than a certain number (here 1200) of NAs
def name_gettr(data, treshold = 1200):
    res = {}
    for el in data[:1]:
        if data[el].isnull().sum() > treshold:
            res[el] = data[el].isnull().sum()
    return res

#apply function defined above
df14_outl = name_gettr(df14)
df15_outl = name_gettr(df15)
df16_outl = name_gettr(df16)
df17_outl = name_gettr(df17)
df18_outl = name_gettr(df18)


#create a list with dictionaries in order to run another function on them 
df_all_outl = [df14_outl, df15_outl, df16_outl, df17_outl, df18_outl]

#function which creates a list with all unique variables that have more than a certain number of NAs for at least one of the years of interest
def unique_outl(data):
    res = []
    for el in data:
        for e in el.keys():
            if not e in res:
                res.append(e)
    return(res)

#apply function and count number of variables to be retired from df
unique_outl(df_all_outl)
len(unique_outl(df_all_outl)) #52 (out of 225, i.e. ~23%) variables


#further ideas if time allows it:
    #graph showing percentages of values missing (see https://www.kaggle.com/mikevasi/risk-aversion-in-the-stock-market, line 4)

['operatingCycle',
 'cashConversionCycle',
 'Net Debt',
 'Other Assets',
 'priceEarningsToGrowthRatio',
 'enterpriseValueMultiple',
 'ebtperEBIT',
 'niperEBT',
 'effectiveTaxRate',
 'nIperEBT',
 'eBTperEBIT',
 'shortTermCoverageRatios',
 'dividendPayoutRatio',
 'Net Debt to EBITDA',
 '10Y Revenue Growth (per Share)',
 '5Y Revenue Growth (per Share)',
 '10Y Operating CF Growth (per Share)',
 '5Y Operating CF Growth (per Share)',
 '10Y Net Income Growth (per Share)',
 '5Y Net Income Growth (per Share)',
 '10Y Shareholders Equity Growth (per Share)',
 '5Y Shareholders Equity Growth (per Share)',
 '10Y Dividend per Share Growth (per Share)',
 'Total non-current assets',
 'Total non-current liabilities',
 'priceBookValueRatio',
 'priceToBookRatio',
 'priceCashFlowRatio',
 'returnOnAssets',
 'returnOnCapitalEmployed',
 'payablesTurnover',
 'cashFlowToDebtRatio',
 'freeCashFlowOperatingCashFlowRatio',
 'cashFlowCoverageRatios',
 'Enterprise Value',
 'PB ratio',
 'PTB ratio',
 'EV to Sales',
 

52

In [122]:
##### implementing a KNN imputation
imputer = KNNImputer(n_neighbors=20, weights='distance', metric='nan_euclidean', copy=True)

#df14.drop(['Sector'], axis = 1, inplace = True)

df14_imputed = imputer.fit_transform(df14)
df14_imputed = pd.DataFrame(df14_imputed)
df14_imputed.columns = list(df14)
df14_imputed.isnull().sum().sum()

0

In [119]:
df14['Sector']

KeyError: 'Sector'

In [111]:
##### inspection of 0-values

df14_zero_vals = df14.isin([0]).sum()
#df14_zero_vals

df15_zero_vals = df15.isin([0]).sum()
#df15_zero_vals

df16_zero_vals = df16.isin([0]).sum()
#df16_zero_vals

df17_zero_vals = df17.isin([0]).sum()
#df17_zero_vals

df18_zero_vals = df18.isin([0]).sum()
#df18_zero_vals


#seq = np.arange(df14.shape[1])
#y = df14_zero_vals.values.tolist()
#plt.figure(figsize=(50,10))
#plt.subplot(2,1,2)
#plt.bar(seq, y)
#plt.ylabel('number of 0-values', fontsize = 50)
#plt.xticks(seq, df14_zero_vals.index.values, rotation = 'vertical')
#plt.show()

In [37]:
##### naive classification tree in order to see whether one of the variables ommitted is able to explain the dependent variable
X = df14.drop(['Class', 'Sector'], axis=1)
Y = df14['Class']

tree = DecisionTreeClassifier(max_depth=100)
tree.fit(X, Y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## Imputing

possible ways of imputing:
* dropna() -> not very elaborate, dropping all NAs would result in a major data waste
* fillna() -> very simplistic method, probably not appropriate
* imputing with mean values -> [...] [T]he correlation structure of a dataset may not be captured if the decision is to plug missing values with zeros or with mean value over the samples. (Imputation of Missing Values in Economic and Financial Time Series Data Using Five Principal Component Analysis (PCA) Approaches, 2019, s. 52)
* imputing with median values
* imputing with most frequent value
* imputing with mean/median/most frequent value of each of the sectors (assuming macro-trends and macro-factors within each sector)
* KNN imputation
* iterative imputer (scikit-learn)


possible papers:
* A review on missing value estimation using imputation algorithm, Journal of Physics ConferenceSeries
* Gautrain, C., & Ravi, V. (2015). Data imputation via evolutionary computation,clustering and a neural network. Neurocomputing, 156:134-142
* Ping, X. O., Lai F., Tseng Y.J, Liang J. D., Huang G. T., Yang, P.M. (2014).Evaluation of imputation methods for missing data and their eﬀect on thereliability of predictive models. International Conference on Bioinformatics,Biocomputational Systems and Biotechnologies, 6, 8-14.
* Schmitt, P; Mandel, J; & Guedj, M. (2015). A comparison of six methods formissing data imputation. Journal of Biometrics & Biostatistics, 6(224), 1-6.
* Imputation of Missing Values in Economic and Financial Time Series Data Using Five Principal Component Analysis (PCA) Approaches, August 2019, Central Bank of Nigeria Journal of Applied Statistics, DOI: 10.33429/Cjas.10119.3/6
* IMPUTATION OF MISSING VALUES IN THE FUNDAMENTAL DATA: UNLEASHING MICE FRAMEWORK 

## Topics to consider
* imputing
    * (e.g. KNN, https://www.kaggle.com/priyankasachdeva20/catboost-model-to-classify-buy-or-sell-stocks/comments line 11)
    * https://scikit-learn.org/stable/modules/impute.html
    * perhaps different methods for different variables
* 0s in df
* outliers
    * Z score
    * IQR



* selection of relevant ratios/fundamental data
    * remove variables which are heavily correlated to each other (see line 7 in https://www.kaggle.com/mikevasi/risk-aversion-in-the-stock-market)
* feature engineering 
    * https://www.nber.org/system/files/working_papers/w25398/w25398.pdf (Dacheng, Empirical Asset Pricing via ML)
    * momentum?
    * categorical variable for market cap
* class imbalance
    * use the stratify option available within sklearn.model_selection.train_test_split
* analysis